In [ ]:
# SIN NUMERO ALEATORIO

def extract_info( df ):
    
    options = Options()
    service = Service( ChromeDriverManager().install( ) )
    driver  = webdriver.Chrome( service = service )
    driver.maximize_window()

    url     = f'https://enlinea.sunedu.gob.pe/'
    driver.get( url )
    wait    = WebDriverWait( driver, 60 )
    
    verificar_titulos_popup = driver.find_element( By.XPATH, '//*[@id="dvEnLinea"]/div[2]/div[3]/div/div[2]/div/a' )
    verificar_titulos_popup.click()
    
    frame_popup = driver.find_element( By.XPATH, '//*[@id="ifrmShowFormConstancias"]' )
    driver.switch_to.frame( frame_popup )
    
    for index, row in df.iterrows():
        
        dni_valor = row[ 'dni' ]
        
        dni_campo = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="doc"]' ) ) )
        dni_campo.clear()
        dni_campo.send_keys( dni_valor )
        
        max_retries = 5

        for retry in range( max_retries ):
        
            try: 

                time.sleep( 2 )
                captcha       = driver.find_element( By.XPATH, '//*[@id="captchaImg"]/img' )
                captcha_image = captcha.screenshot_as_png
                with open( 'image.png', 'wb' ) as f:
                    f.write( captcha_image )

                clave_valor   = pytesseract.image_to_string( Image.open( 'image.png' ),
                                                             config = '-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789' ).strip()
                campo_captcha = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="captcha"]' ) ) )
                campo_captcha.clear()
                campo_captcha.send_keys( clave_valor )                

                buscar_boton = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="buscar"]' ) ) )
                buscar_boton.click()
                
                print( f'Clave: { clave_valor }' )
                break
                
            except Exception as e:
                
                print( f'Error: { e }' )
                time.sleep( 5 )
                boton_x = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="closeModalError"]' ) ) )
                boton_x.click()
        
        grado         = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[2]/b[1]/p' ) ) ).text
        fecha_diploma = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[2]/b[2]' ) ) ).text
        modalidad     = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[2]/span' ) ) ).text
        institucion   = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[3]/span' ) ) ).text
        
        print( f'Grado: { grado }' )
        print( f'Institutción: { institucion }' )
        
        df.at[ index, 'grado' ]         = grado       
        df.at[ index, 'fecha_diploma' ] = fecha_diploma    
        df.at[ index, 'modalidad' ]     = modalidad    
        df.at[ index, 'institucion' ]   = institucion
        
        # Salir del iframe para recargar la página principal
        driver.switch_to.default_content()

        # Recargar la página principal
        driver.refresh()

        # Volver a entrar en el iframe
        verificar_titulos_popup = driver.find_element( By.XPATH, '//*[@id="dvEnLinea"]/div[2]/div[3]/div/div[2]/div/a' )
        verificar_titulos_popup.click()
        frame_popup = driver.find_element(By.XPATH, '//*[@id="ifrmShowFormConstancias"]')
        driver.switch_to.frame(frame_popup)

        time.sleep(10)
        print( '\n' )
        
    driver.quit()
        
    return df